# 1. INSTALACIÓN DE DEPENDENCIAS

In [1]:
import subprocess
import sys

packages = [
    'langchain-core',
    'langchain-community',
    'chromadb',
    'langchain-ollama'
]

for package in packages:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", package])

# 2. IMPORTACIONES

In [2]:
import json
from langchain_core.documents import Document
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import OllamaEmbeddings
from langchain_ollama import OllamaLLM
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

c:\Users\j-h-o\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 3. CARGAR DATOS DESDE JSON

In [4]:
# Cargar el archivo JSON
with open("data/chunks/chunks_celsia.json", "r", encoding="utf-8") as f:
    datos = json.load(f)

# Extraer chunks (el JSON es una lista de objetos con clave "chunk")
if isinstance(datos, list):
    chunks = [item["chunk"] for item in datos]
else:
    chunks = datos["chunks"]

print(f"Se cargaron {len(chunks)} chunks")

Se cargaron 375 chunks


# 4. CREAR DOCUMENTOS PARA LANGCHAIN

In [ ]:
# Convertir cada chunk en un documento de LangChain
documentos = [
    Document(page_content=chunk, metadata={"source": f"chunk_{i}"})
    for i, chunk in enumerate(chunks)
]

print(f"Se crearon {len(documentos)} documentos")

# 5. CONFIGURAR EMBEDDINGS CON OLLAMA

In [ ]:


# Crear embeddings usando nomic-embed-text con Ollama
embeddings = OllamaEmbeddings(
    model="nomic-embed-text",
    base_url="http://localhost:11434"
)

print("✅ Embeddings configurados")




# 6. CREAR Y GUARDAR VECTORSTORE EN CHROMADB

In [ ]:


# Crear vectorstore y persistir en disco
vectorstore = Chroma.from_documents(
    documents=documentos,
    embedding=embeddings,
    persist_directory="./chromadb_storage",
    collection_name="rag_collection"
)

print(f"ChromaDB guardado en ./chromadb_storage")

# 7. CREAR RETRIEVER

In [ ]:
# Configurar retriever para recuperar los 3 documentos más similares
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3}
)

print("Retriever configurado")

# 8. CARGAR MODELO LLM GEMMA 3 4B

In [ ]:
# Inicializar Gemma 3 4B desde Ollama
llm = OllamaLLM(
    model="gemma3:4b",
    base_url="http://localhost:11434",
    temperature=0.7
)

print("Gemma 3 4B cargado")


✅ Dependencias instaladas


c:\Users\j-h-o\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ Importaciones realizadas
✅ Se cargaron 375 chunks
✅ Se crearon 375 documentos
✅ Embeddings configurados


C:\Users\j-h-o\AppData\Local\Temp\ipykernel_26580\2327974608.py:73: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the `langchain-ollama package and should be used instead. To use it run `pip install -U `langchain-ollama` and import as `from `langchain_ollama import OllamaEmbeddings``.
  embeddings = OllamaEmbeddings(


✅ ChromaDB guardado en ./chromadb_storage
✅ Retriever configurado
✅ Gemma 3 4B cargado
✅ Prompt personalizado creado
✅ Cadena RAG creada

❓ Pregunta: ¿Qué oportunidades de inversión mencionas?

✅ Respuesta:
El texto menciona varias oportunidades de inversión:

*   **Mecanismo de Readquisición de Acciones (hashtag):** Celsia y Talentlab Group participan activamente en este mecanismo, invirtiendo impuestos en proyectos de impacto económico y social en municipios ZOMAC y en el desarrollo de DE&I.
*   **Proyectos de Celsia:** Inversión en obras viales, proyectos de educación, recuperación de bosques y construcción de infraestructura en Tolima, Valle, Antioquia y Cauca.
*   **Talentlab Group:** Participación en el ranking de DE&I, lo que representa una inversión en el desarrollo de la cultura organizacional y la innovación.
*   **Proyectos de Generación y Transmisión de Energía:** Especialmente eólicos en La Guajira.

📚 Documentos recuperados:

[1] chunk_147:
En nuestra “Colombia tierra que

# 9. CREAR PROMPT 

In [ ]:
# Definir el template del prompt
prompt_template = """**[INSTRUCCIONES CLAVE ZERO-SHOT Y LIMITACIÓN DE FUENTE]**
Tu ÚNICA tarea es responder a la **PREGUNTA** del usuario, utilizando EXCLUSIVAMENTE la información que se encuentra en el **CONTEXTO** proporcionado a continuación.

**REGLAS ESTRICTAS para evitar alucinaciones:**
1.  **SI** la respuesta a la PREGUNTA se encuentra explícita o implícitamente en el **CONTEXTO**, genera una respuesta completa y profesional.
2.  **SI** no puedes encontrar la respuesta en el **CONTEXTO**, o si la información es insuficiente, debes responder **ÚNICAMENTE** con la siguiente frase predefinida: "Lamento no poder ofrecer una respuesta precisa basada en la información disponible. Por favor, consulta los canales oficiales de CELSIA o llama a la línea de servicio al cliente."
3.  **NUNCA** utilices tu conocimiento general o información que no esté en el **CONTEXTO**. **NUNCA** inventes tarifas, fechas o procesos.
Coloca el cursor sobre un mensaje para fijarlo

Contexto:
{context}

Pregunta: {question}

Respuesta:"""

prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"]
)

print("Prompt personalizado creado")


✅ Prompt personalizado creado
✅ Cadena RAG creada

❓ Pregunta: ¿Qué es Celsia

✅ Respuesta:
Celsia es una empresa pionera en soluciones integrales de Eficiencia Energética con más de 7 años de experiencia y más de 500 clientes que la respaldan.


📚 Documentos recuperados:

[1] chunk_222:
En Celsia somos pioneros en 𝘀𝗼𝗹𝘂𝗰𝗶𝗼𝗻𝗲𝘀 𝗶𝗻𝘁𝗲𝗴𝗿𝗮𝗹𝗲𝘀 𝗱𝗲 𝗘𝗳𝗶𝗰𝗶𝗲𝗻𝗰𝗶𝗮 𝗘𝗻𝗲𝗿𝗴𝗲́𝘁𝗶𝗰𝗮 con más de 𝟳 𝗮ñ𝗼𝘀 𝗱𝗲 𝗲𝘅𝗽𝗲𝗿𝗶𝗲𝗻𝗰𝗶𝗮 y más de 500 clientes que lo respaldan.💡👌🏻 ⚡𝗖𝗼𝗻𝗼𝗰𝗲 𝗲 𝗶𝗺𝗽𝗹𝗲𝗺𝗲𝗻𝘁𝗮 𝗘𝗳𝗶𝗰𝗶𝗲𝗻𝗰𝗶𝗮 𝗘𝗻𝗲𝗿𝗴𝗲́𝘁𝗶𝗰𝗮 con Celsia.com⚡


[2] chunk_288:
Edición 10 Tolima. En esta edición puedes encontrar:En las tiendas Celsia abrimos los sábados.Te explicamos por qué se cobra el servicio de aseo en tu factura.Ideas para ahorrar energía en esta Navidad.


[3] chunk_163:
El 5 de noviembre presentamos los hashtag y en este post te vamos a compartir algunas de las respuestas que dieron en la teleconferencia nuestros líderes RICARDO ANDRES SIERRA FERNANDEZ, Carlos Solano (regulación) y Esteban Piedrahita (financiera). ✅ Situac

# 10. CREAR CADENA RAG (LANGCHAIN RUNNABLE)

In [ ]:


# Función para formatear documentos recuperados
def formato_docs(docs):
    return "\n\n".join([doc.page_content for doc in docs])

# Crear cadena RAG usando la API  de LangChain
rag_chain = (
    {"context": retriever | formato_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
)

print("Cadena RAG creada")


# 11. REALIZAR CONSULTA

In [ ]:
# Ejemplo de consulta
pregunta = "¿Qué es Celsia?"

# Invocar la cadena RAG
respuesta = rag_chain.invoke(pregunta)

# Mostrar respuesta
print(f"Respuesta:\n{respuesta}\n")

# Mostrar documentos recuperados
docs_recuperados = retriever.invoke(pregunta)
print("Documentos recuperados:")
for i, doc in enumerate(docs_recuperados, 1):
    print(f"\n[{i}] {doc.metadata['source']}:\n{doc.page_content}\n")